<a href="https://colab.research.google.com/github/ziatdinovmax/gpax/blob/main/examples/gpax_GPBO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gaussian process-based Bayesian optimization

*Prepared by Maxim Ziatdinov (2022). Last updated in October 2023.*

As described in earlier examples, Gaussian process (GP) is a powerful tool for reconstructing an unknown function from sparse measurements in the probabilistic fashion. In addition to providing a "one-off" reconstruction, the GP's posterior predictive mean and uncertainty can be used to derive an acquisition function for selecting the next point to measure in the optimization problems. In the fully Bayesian regime, the next measurement point is selected according to

$$ x_{next}= \underset{x}{\arg\max}\frac{1}{L}∑_{i=1}^Lα(𝜇_*^i,𝓥_*^i)\qquad (1) $$

where $L$ is the total number of Hamiltonian Monte Carlo samples,  $𝜇_*^i$ is a posterior predictive mean, and $𝓥_*^i$ is a posterior predictive variance for *i*-th sample. Perhaps the simplest acquisition function is an upper confidence bound (UCB) defined as

$$ α_{UCB}^i= 𝜇_*^i\pm\sqrt{𝛽\space 𝓥_*^i}\qquad (2) $$

where the square root of $𝓥_*$ is a standard deviation (‘uncertainty’). The coefficient $𝛽$ determines an exploitation-exploration trade-off. The '$+$' sign corresponds to the maximization problems, whereas the '$-$' sign is for the minimization problems. In the variational inference regime, $L=1$, and the next point is selected as $x_{next}=\arg\max_xα(𝜇_*,𝓥_*)$.

For the analytical acquisition function such as UCB, one may alternatively first derive mean and variance of the sampled predictions (```y_sampled``` in GPax code examples) and then use them to compute the acqusition function.


## Install & Import

Install the latest GPax package from PyPI (this is best practice, as it installs the latest, deployed and tested version).

In [ ]:
!pip install gpax

Import needed packages:

In [ ]:
try:
    # For use on Google Colab
    import gpax

except ImportError:
    # For use locally (where you're using the local version of gpax)
    print("Assuming notebook is being run locally, attempting to import local gpax module")
    import sys
    sys.path.append("..")
    import gpax

In [ ]:
import numpy as np
import numpyro
import matplotlib.pyplot as plt

gpax.utils.enable_x64()  # enable double precision

Enable some pretty plotting.

In [ ]:
import matplotlib as mpl

In [ ]:
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
mpl.rcParams['text.usetex'] = False
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)
mpl.rcParams['figure.dpi'] = 200

# Sequential Gaussian process-based Bayesian optimization

Let's define a function to be minimized and a function that emulates a noisy measurement:

In [ ]:
def func(x, y=1.2):
    out = (
        -20 * np.exp(-0.2 * np.sqrt(0.5 * (x**2 + y**2)))
        - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(2 * np.pi * y)))
        + np.e + 20
    )
    return out

def measure(x, noise=0.1):
    return func(x) + noise * np.random.randn(len(x))

Next, we generate a few noisy observations of our function. We also plot the true function ("ground truth") to confirm the location of the minimum at $x=0$ but we are not going to use it anywhere.

In [ ]:
np.random.seed(42)

X_bounds = np.array([-2, 2])
X = np.random.uniform(X_bounds[0], X_bounds[1], size=(8,))
X = np.append(X, X_bounds)
X = np.sort(X)
y = measure(X)

X_unmeasured = np.linspace(X_bounds[0], X_bounds[1], 200)
ground_truth = measure(X_unmeasured, noise=0)


_, ax = plt.subplots(figsize=(4, 2))
ax.set_xlabel("$X$", fontsize=16)
ax.set_ylabel("$y$", fontsize=16)
ax.scatter(X, y, marker='x', c='k', s=64, zorder=1, label="Observations", alpha=1.0)
ax.plot(X_unmeasured, ground_truth, label='Ground truth')
ax.legend(loc='best');

The Gaussian process class in GPax uses a weakly informative $LogNormal(0,1)$ prior distribution for all kernel parameters and model noise by default. If we have prior knowledge that the noise level is low, we may choose a more appropriate prior distibution for the noise, such as

In [ ]:
noise_prior = numpyro.distributions.HalfNormal(0.01)

Next we define a single step that takes measured data, trains a GP model, and uses it to compute an Upper Confidence Bound (UCB) acquisition function for deriving the next measurment point (which will be done inside the main loop).

In [ ]:
def step(X_measured, y_measured, X_unmeasured):
    
    # Get random number generator keys for training and prediction
    rng_key1, rng_key2 = gpax.utils.get_keys()
    
    # Initialize GP model
    gp_model = gpax.ExactGP(1, kernel='RBF', noise_prior_dist=noise_prior)
    
    # Run HMC to obtain posterior samples for the GP model parameters
    gp_model.fit(rng_key1, X_measured, y_measured)
    
    # Get predictions (we don't need this step for optimization -
    # only for visualization purposes)
    y_pred, y_sampled = gp_model.predict(
        rng_key2,
        X_unmeasured,
        noiseless=True
    )
    
    # Compute acquisition function
    obj = gpax.acquisition.UCB(
        rng_key2,
        gp_model,
        X_unmeasured,
        beta=4,
        maximize=False,
        noiseless=True
    )

    return obj, (y_pred, y_sampled)

Finally, we run the Bayesian optimization for 7 steps to find the minimum of the unknown (to the algorithm) function:

In [ ]:
num_steps = 7

fig, axs = plt.subplots(num_steps, 2, figsize=(5, 8), sharex=True)

axs[0, 0].set_title("Data & GP")
axs[0, 1].set_title("Acquisition Function")

for e in range(num_steps):
    ax = axs[e]

    print(f"\nStep {e+1}/{num_steps}")

    # Compute acquisition function
    acq, (y_pred, y_sampled) = step(X, y, X_unmeasured)

    # Get the next point to evaluate
    idx = acq.argmax()
    next_point = X_unmeasured[idx:idx+1]

    # Measure the point
    next_point_value = measure(next_point)

    # Update measured data
    X = np.append(X, X_unmeasured[idx:idx+1])
    y = np.append(y, next_point_value)

    # Plot observed points, mean prediction, and acqusition function
    lower_b = y_pred - y_sampled.std(axis=(0,1))
    upper_b = y_pred + y_sampled.std(axis=(0,1))

    ax1 = ax[0]
    ax2 = ax[1]
    
    ax1.scatter(X[:-1], y[:-1], marker='x', c='k', label="Observations", s=64)
    ax1.plot(X_unmeasured, y_pred, lw=1, c='b', label='Posterior mean')
    ax1.fill_between(
        X_unmeasured,
        lower_b,
        upper_b,
        color='blue',
        alpha=0.3,
        linewidth=0,
        label="Model uncertainty", 
    )
    
    ax2.plot(X_unmeasured, acq, lw=2, c='orangered', label='Acquisition function')
    ax2.scatter(
        X_unmeasured[idx],
        acq[idx],
        s=30,
        marker="o",
        color="black",
        label='Next point to measure',
        zorder=3,
    )
    ax1.text(
        0.02, 0.02, f"{e+1}/{num_steps}", ha="left", va="bottom", transform=ax1.transAxes
    )

plt.subplots_adjust(wspace=0.3)
plt.show()

As one can see, the algorithm quickly converged onto the true minimum. Note that in real experiments, it's practical to update the ```X_unmeasured``` at each step by removing the just measured point from it.